# 过滤数据

In [1]:
import numpy as np
import pandas as pd 
from pathlib import Path
import os 
from PIL import Image
import shutil
from glob import glob
from tqdm import tqdm 
from joblib import Parallel, delayed
from typing import Union
from torchvision.io import ImageReadMode, read_image
from transformers import ChineseCLIPProcessor, ChineseCLIPModel

In [2]:
all_image_path_list = glob("bigdata/image_data/*")#[:20]
all_image_path_list[:2]

['bigdata/image_data\\test-0.jpg', 'bigdata/image_data\\test-1.jpg']

In [3]:
model_name_or_path = "OFA-Sys/chinese-clip-vit-large-patch14"
processor = ChineseCLIPProcessor.from_pretrained(model_name_or_path)

In [11]:
def filter_img(x: str) -> Union[str, None]:
    try:
        image = Image.open(x)
        processor(images=image, return_tensors="pt")
        # value = read_image(x, mode=ImageReadMode.RGB)
        # if len(value.shape) ==3:
        return x

    except Exception as e:
        print(e)
        return None

# read_image(all_image_path_list[0], mode=ImageReadMode.RGB).shape
# clean_image_path_list = [filter_img(i) for i in tqdm(all_image_path_list[:100])]
clean_image_path_list = Parallel(n_jobs=-1, verbose=10)(delayed(filter_img)(i) for i in all_image_path_list)
len(clean_image_path_list)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0191s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 106 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1950s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 170 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 204 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 244 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 320 tasks      | elapsed:    1.0s
[Parallel(n_j

1370872

In [12]:
clean_image_path_list = [i for i in clean_image_path_list if i is not None]
len(clean_image_path_list)

1370013

In [14]:
os.makedirs("bigdata/temp_dir", exist_ok=True)
pd.DataFrame({'image_path':clean_image_path_list}).to_csv("bigdata/temp_dir/temp_image_url.csv", index=False)